In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
승하차_파일 = "../../data/결과/승하차/통합/2호선_승하차인원_통합.csv"
혼잡도_파일 = "../../data/결과/혼잡도/통합/2호선_혼잡도_통합.csv"
시간표_파일 = "../../data/결과/운행시간표/통합/2호선_열차운행시각표.csv"

승하차_df = pd.read_csv(승하차_파일, encoding="euc-kr")
혼잡도_df = pd.read_csv(혼잡도_파일, encoding="euc-kr")
시간표_df = pd.read_csv(시간표_파일, encoding="euc-kr")

In [8]:
def 실제값_단순확인(역명, 요일, 시간):
    승차 = 승하차_df[
        (승하차_df["역명"] == 역명)
        & (승하차_df["평일주말"] == 요일)
        & (승하차_df["구분"] == "승차")
    ]
    하차 = 승하차_df[
        (승하차_df["역명"] == 역명)
        & (승하차_df["평일주말"] == 요일)
        & (승하차_df["구분"] == "하차")
    ]
    혼잡도 = 혼잡도_df[
        (혼잡도_df["역명"] == 역명)
        & (혼잡도_df["평일주말"] == 요일)
        & (혼잡도_df["구분"] == "상행")
    ]
    print("🏃 승차 인원:", int(승차[시간].values[0]) if not 승차.empty else "없음")
    print("🚶 하차 인원:", int(하차[시간].values[0]) if not 하차.empty else "없음")
    print("📊 혼잡도:", float(혼잡도[시간].values[0]) if not 혼잡도.empty else "없음")


# 예시 실행
실제값_단순확인("신도림", "평일", "17:00")

🏃 승차 인원: 4376
🚶 하차 인원: 4200
📊 혼잡도: 37.3


In [9]:
# 분석 대상 설정
target_station = "신도림"
target_daytype = "평일"
target_direction = "상행"  # '상행', '하행' 또는 필요에 맞게
time_cols = [col for col in 승하차_df.columns if ":" in col]

result_rows = []
for tcol in time_cols:
    row = {}
    # 승차 찾기
    승차 = 승하차_df[
        (승하차_df["역명"] == target_station) &
        (승하차_df["평일주말"] == target_daytype) &
        (승하차_df["구분"] == "승차")
    ]
    승차값 = int(승차[tcol].values[0]) if len(승차) > 0 else 0
    # 하차 찾기
    하차 = 승하차_df[
        (승하차_df["역명"] == target_station) &
        (승하차_df["평일주말"] == target_daytype) &
        (승하차_df["구분"] == "하차")
    ]
    하차값 = int(하차[tcol].values[0]) if len(하차) > 0 else 0
    # 혼잡도 찾기
    혼잡도 = 혼잡도_df[
        (혼잡도_df["역명"] == target_station) &
        (혼잡도_df["평일주말"] == target_daytype) &
        (혼잡도_df["구분"] == target_direction)
    ]
    혼잡도값 = float(혼잡도[tcol].values[0]) if len(혼잡도) > 0 else np.nan
    # 레코드 저장
    row["시간"] = tcol
    row["승차인원"] = 승차값
    row["하차인원"] = 하차값
    row["혼잡도"] = 혼잡도값
    result_rows.append(row)

final_df = pd.DataFrame(result_rows)
final_df["시간_int"] = final_df["시간"].apply(lambda x: int(x.split(":")[0]))
final_df = final_df.sort_values("시간_int").reset_index(drop=True)
print(final_df.shape)
print(final_df.head())


(20, 5)
     시간  승차인원  하차인원    혼잡도  시간_int
0  0:00   301  1758    0.0       0
1  5:00  1338   175   11.2       5
2  6:00  2165   677   27.0       6
3  7:00  5911  2021   55.9       7
4  8:00  5968  5723  100.0       8


In [10]:
from sklearn.preprocessing import MinMaxScaler

features = final_df[["승차인원", "하차인원", "혼잡도"]].fillna(0).values
target = final_df["승차인원"].values  # 예측할 목표값

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)


In [14]:
def create_sequences(data, target, seq_length=4):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:(i + seq_length)])
        ys.append(target[i + seq_length])
    return np.array(xs), np.array(ys)

X, y = create_sequences(features_scaled, target, seq_length=8)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(16, input_shape=(X.shape[1], X.shape[2])),
    Dense(8, activation="relu"),
    Dense(1)
])
model.compile(loss="mse", optimizer="adam")
model.fit(X, y, epochs=5000, batch_size=10, verbose=30)


Epoch 1/5000


C:\Users\user\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 2/5000
Epoch 3/5000
Epoch 4/5000
Epoch 5/5000
Epoch 6/5000
Epoch 7/5000
Epoch 8/5000
Epoch 9/5000
Epoch 10/5000
Epoch 11/5000
Epoch 12/5000
Epoch 13/5000
Epoch 14/5000
Epoch 15/5000
Epoch 16/5000
Epoch 17/5000
Epoch 18/5000
Epoch 19/5000
Epoch 20/5000
Epoch 21/5000
Epoch 22/5000
Epoch 23/5000
Epoch 24/5000
Epoch 25/5000
Epoch 26/5000
Epoch 27/5000
Epoch 28/5000
Epoch 29/5000
Epoch 30/5000
Epoch 31/5000
Epoch 32/5000
Epoch 33/5000
Epoch 34/5000
Epoch 35/5000
Epoch 36/5000
Epoch 37/5000
Epoch 38/5000
Epoch 39/5000
Epoch 40/5000
Epoch 41/5000
Epoch 42/5000
Epoch 43/5000
Epoch 44/5000
Epoch 45/5000
Epoch 46/5000
Epoch 47/5000
Epoch 48/5000
Epoch 49/5000
Epoch 50/5000
Epoch 51/5000
Epoch 52/5000
Epoch 53/5000
Epoch 54/5000
Epoch 55/5000
Epoch 56/5000
Epoch 57/5000
Epoch 58/5000
Epoch 59/5000
Epoch 60/5000
Epoch 61/5000
Epoch 62/5000
Epoch 63/5000
Epoch 64/5000
Epoch 65/5000
Epoch 66/5000
Epoch 67/5000
Epoch 68/5000
Epoch 69/5000
Epoch 70/5000
Epoch 71/5000
Epoch 72/5000
Epoch 73/5000


In [24]:
def predict_station_passenger(model, scaler, df, input_time, seq_len=4):
    시간_int = int(input_time.split(":")[0])
    idx = df[df["시간_int"] == 시간_int].index
    if len(idx) == 0 or idx[0] < seq_len:
        return "해당 시각은 예측 불가(데이터 부족)"
    idx = idx[0]
    x_input = df[["승차인원", "하차인원", "혼잡도"]].fillna(0).values
    x_input = scaler.transform(x_input)
    x_input = x_input[idx - seq_len: idx].reshape(1, seq_len, 3)
    pred = model.predict(x_input)
    return f"{input_time} 기준 예상 승차인원: {int(pred[0,0])}명"

# 사용 예시
print(predict_station_passenger(model, scaler, final_df, "17:00"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
17:00 기준 예상 승차인원: 3262명
